In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bsoup
import wikipedia as wk

In [2]:
page='https://en.wikipedia.org/wiki/List_of_Indian_Americans'

In [4]:
wpage = wk.WikipediaPage(title='List of Indian Americans')
wpage

<WikipediaPage 'List of Indian Americans'>

In [6]:
raw_str = wpage.content
str_list = raw_str.split('\n')
str_list

['This is a list of notable Indian Americans, including both original immigrants who obtained U.S. citizenship and their American descendants. Note that unlike many countries, India does not allow dual citizenship.',
 'To be included in this list, the person must have a Wikipedia article showing they are Indian American or must have references showing they are Indian American and are notable.',
 '',
 '',
 '== Academics ==',
 '',
 '',
 '=== Nobel Prize recipients ===',
 '',
 'Har Gobind Khorana, Nobel Prize in Medicine, 1968',
 'Subramanyan Chandrasekhar, Nobel Prize for Physics, 1983',
 'Amartya Sen, Nobel Memorial Prize in Economic Sciences in 1998; Professor of Economics and Philosophy at Harvard University',
 'Venkatraman Ramakrishnan, Nobel Prize in Chemistry, 2009; current President of the Royal Society (as of November 2015)',
 '',
 '',
 '=== Deans and presidents ===',
 'Rakesh Khurana, Dean of Harvard College',
 'Jamshed Bharucha, President Emeritus of Cooper Union, previously De

In [7]:
len(str_list)

732

In [12]:
str_list[2] == ''

True

In [13]:
str_list2 = []
for s in str_list:
    if not s=='':
        str_list2.append(s)
len(str_list2)

644

In [18]:
str_list3=[]
for s in str_list2:
    if not s.startswith('='):
        str_list3.append(s)
len(str_list3)

609

In [ ]:
str_list3.pop(0)
str_list3.pop(0)

In [24]:
str_list3

['Har Gobind Khorana, Nobel Prize in Medicine, 1968',
 'Subramanyan Chandrasekhar, Nobel Prize for Physics, 1983',
 'Amartya Sen, Nobel Memorial Prize in Economic Sciences in 1998; Professor of Economics and Philosophy at Harvard University',
 'Venkatraman Ramakrishnan, Nobel Prize in Chemistry, 2009; current President of the Royal Society (as of November 2015)',
 'Rakesh Khurana, Dean of Harvard College',
 'Jamshed Bharucha, President Emeritus of Cooper Union, previously Dean of Arts & Sciences at Dartmouth College and Provost at Tufts University',
 'Vijay K. Dhir, Dean of the UCLA Henry Samueli School of Engineering and Applied Science',
 "Dinesh D'Souza, Former president of The King's College, New York",
 'Anjli Jain, Executive Director of CampusEAI Consortium',
 'Dipak C. Jain, Dean of INSEAD and former Dean of the Kellogg School of Management at Northwestern University',
 'Vistasp Karbhari, President of the University of Texas at Arlington',
 'Pramod Khargonekar, control theorist;

In [43]:
dict_list=[]
for s in str_list3:
    parts = s.split(',')
    dict1 = {'name':parts[0],
            'info':parts[1] if len(parts)>1 else None}
    dict_list.append(dict1)

## throw this into a dataframe

In [44]:
df = pd.DataFrame(dict_list, columns=['name','info'])
df.head()

,name,info
0,Har Gobind Khorana,Nobel Prize in Medicine
1,Subramanyan Chandrasekhar,Nobel Prize for Physics
2,Amartya Sen,Nobel Memorial Prize in Economic Sciences in ...
3,Venkatraman Ramakrishnan,Nobel Prize in Chemistry
4,Rakesh Khurana,Dean of Harvard College


Save this to a CSV

In [47]:
df.to_csv('names.csv')

In [63]:
df.iloc[606]['info']==None

True

In [69]:
df[df['info'].isnull()]

,name,info
52,Raghuram Rajan Professor University of Chicago...,None
232,Aziz Ansari,None
233,Aman Ali,None
234,Arj Barker,None
235,Jay Chandrasekhar,None
236,Hari Kondabolu,None
237,Adam Mamawala,None
239,Rajiv Satyal,None
240,Anish Shah,None
241,Azhar Usman,None


In [73]:
df.drop([605, 606], inplace=True)

In [74]:
df.shape

(605, 2)

## Time to get ethnicity

In [82]:
import requests
from urllib.request import quote
prism_url = 'http://www.name-prism.com/api_eth/json/'

In [89]:
query = 'Atma Bharathi Mani'
resp = requests.get(prism_url + quote(query))

In [95]:
pp = resp.json()['2PRACE']
round(pp*100, 2)

0.23

In [96]:
resp.json()

{'2PRACE': 0.0023401270293412603,
 'Hispanic': 0.006478390665164317,
 'API': 0.942312085963781,
 'Black': 0.0005973619951487627,
 'AIAN': 0.0003318037971566136,
 'White': 0.04794023054940803}

In [130]:
import time
for index, row in df.iterrows():
    resp = requests.get(prism_url + quote(row['name']))

    if resp.status_code == 200:
        # parse response
        eth_json = resp.json()
        
        # add columns
        df.at[index,'multi_race'] = round(eth_json['2PRACE']*100,2)
        df.at[index,'asian'] = round(eth_json['API']*100,2)
        df.at[index,'hispanic'] = round(eth_json['Hispanic']*100,2)
        df.at[index,'black'] = round(eth_json['Black']*100,2)
        df.at[index,'white'] = round(eth_json['White']*100,2)
        df.at[index,'native_american'] = round(eth_json['AIAN']*100,2)

        print('processed: ' + row['name'])
    time.sleep(1)

processed: Har Gobind Khorana
processed: Subramanyan Chandrasekhar
processed: Amartya Sen
processed: Venkatraman Ramakrishnan
processed: Rakesh Khurana
processed: Jamshed Bharucha
processed: Vijay K. Dhir
processed: Dinesh D'Souza
processed: Anjli Jain
processed: Dipak C. Jain
processed: Vistasp Karbhari
processed: Pramod Khargonekar
processed: Renu Khator
processed: Pradeep Khosla
processed: Vijay Kumar
processed: Geeta Menon
processed: Nitin Nohria
processed: Sethuraman Panchanathan
processed: Michael Rao
processed: Beheruz Sethna
processed: Paul Shrivastava
processed: Molly Easo Smith
processed: Kumble R. Subbaswamy
processed: Subra Suresh
processed: Satish K. Tripathi
processed: Sundaraja Sitharama Iyengar
processed: S. Shankar Sastry
processed: Shreeram Shankar Abhyankar
processed: Raghu Raj Bahadur
processed: Manjul Bhargava
processed: Raj Chandra Bose
processed: Sarvadaman Chowla
processed: Harish-Chandra
processed: Narendra Karmarkar
processed: Chandrashekhar Khare
processed: G

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [134]:
resp.text

'Exceed request limits. See page "API" to resolve.'

In [136]:
index

48

In [137]:
df.to_csv('names.csv')